INPUTS: Take in extracted data from step 1

Apart from normal cleanup to standardize spellings and capitalizations, we are also imposing an alphabetical ordering where pigment1 is always first alphabetically and pigment2 is always second.

The reason for this is for acheivements / history data, it provides a stable key so that I can map people (regardless of what order they submit pigments in) to existing pigment data.

In [4]:
// Import necessary packages
#r "nuget: FSharp.Data"
open FSharp.Data

// Load JSON using FSharp.Data
type ComboJson = JsonProvider<"../data/extracted/ExtractedPigments-2025-08-30.json">
let rawData = ComboJson.Load("../data/extracted/ExtractedPigments-2025-08-30.json")

// Define cleanup functions
let cleanupPigmentName name =
    if name = "Phlox Paroxysm" then "Phylox Paroxysm"
    else name

let cleanupResultName (name: string) =
    if name.ToLower() = "scene: village within a time pocket" then
        "scene: Village in a Time Pocket"
    else name

// Apply cleanup and alphabetical reordering
let cleanedCombos =
    rawData
    |> Array.map (fun c -> 
        let p1 = cleanupPigmentName c.Pigment1
        let p2 = cleanupPigmentName c.Pigment2
        let result = cleanupResultName c.Result
        let pigment1, pigment2 = if p1.CompareTo(p2) <= 0 then p1, p2 else p2, p1
        {| pigment1 = pigment1; pigment2 = pigment2; result = result |}
    )
    |> Array.toList
    |> List.sortBy (fun x -> x.pigment1)

// Output cleaned and reordered data
cleanedCombos


Installed Packages FSharp.Data, 6.6.0

[ { pigment1 = "Ancient Argent"
 pigment2 = "Corrupted Vermillion"
 result = "Abstract Mess" }, { pigment1 = "Gormenghast Moss"
 pigment2 = "Nattier"
 result = "Scene: Countryside Railroad" } ] HeadOrDefault { pigment1 = "Ancient Argent"\n pigment2 = "Corrupted Vermillion"\n result = "Abstract Mess" } pigment1 Ancient Argent pigment2 Corrupted Vermillion result Abstract Mess TailOrNull [ { pigment1 = "Gormenghast Moss"
 pigment2 = "Nattier"
 result = "Scene: Countryside Railroad" } ] HeadOrDefault { pigment1 = "Gormenghast Moss"\n pigment2 = "Nattier"\n result = "Scene: Countryside Railroad" } pigment1 Gormenghast Moss pigment2 Nattier result Scene: Countryside Railroad TailOrNull [ ] HeadOrDefault <null> TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4319\r\n at lambda_method97(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVal... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663785 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\Liam\.nuget\packages\microsoft.dotnet-interactive\1.0.632301\tools\net9.0\any\FSharp.Core.dll ModuleVersionId 1023697b-e00e-d2ad-b568-6f799f8eeeea MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=9.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140735413936240 Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType T ReturnTypeCustomAttributes T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] ReturnParameter T ParameterType T Name <null> HasDefaultValue False DefaultValue RawDefaultValue MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes (empty) Message The input list was empty. Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4319
 at lambda_method97(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4324\r\n at lambda_method98(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVal... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Name get_Tail DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663786 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\Liam\.nuget\packages\microsoft.dotnet-interactive\1.0.632301\tools\n

In [5]:
open System.Text.Json
open System.IO

// Serialize cleaned combos to JSON
let jsonOptions = JsonSerializerOptions(WriteIndented = true)
let json = JsonSerializer.Serialize(cleanedCombos, jsonOptions)

// Ensure the directory exists
let outputDir = "../data/cleaned"
Directory.CreateDirectory(outputDir) |> ignore

// Write to file
let outputPath = Path.Combine(outputDir, "cleaned.json")
File.WriteAllText(outputPath, json)

printfn "✅ Cleaned data written to %s" outputPath


✅ Cleaned data written to ../data/cleaned\cleaned.json
